#신한카드 데이터 이용

In [ ]:
#아래 분석은 구글의 colab을 이용해서 작성했습니다.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
os.chdir("/content/gdrive/My Drive/공모전")

In [ ]:
korea = pd.read_csv('NATIVE_RE.txt', sep = '|')
korea.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,v1,v2,v3,gb3,gb2,sex_ccd,cln_age_r,ta_ym,daw_ccd_r,apv_ts_dl_tm_r,vlm,usec
0,세종,세종,.,취미오락,외식,M,30,201701,RED,활동,351296766,19010
1,세종,세종,.,취미오락,종합쇼핑,M,50,201701,RED,휴식,72244196,2375
2,세종,세종,.,취미오락,종합쇼핑,M,30,201701,WHITE,휴식,532911085,19979
3,충남,세종,.,취미오락,종합쇼핑,M,50,201701,WHITE,활동,50972534,951
4,충남,세종,.,취미오락,종합쇼핑,M,40,201701,RED,휴식,19104457,408


In [ ]:
#MZ세대 뽑아보기
korea_yb = korea[(korea['cln_age_r'] == 30.) | (korea['cln_age_r'] == 20.) ]
#할당된 기간에 근거한 값 뽑아보기(코로나 이후 시대에 데이터만 파악하기 위해서)
korea_yb_myterm = korea_yb[(korea_yb['ta_ym'] >= 202002)]
#편하게 보기 위해서 v2, v3합쳐준 v4변수 만들어주기
korea_yb_myterm['v4'] = korea_yb_myterm['v2'] + ' ' + korea_yb_myterm['v3']
korea_yb_myterm.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,v1,v2,v3,gb3,gb2,sex_ccd,cln_age_r,ta_ym,daw_ccd_r,apv_ts_dl_tm_r,vlm,usec,v4
38034879,세종,세종,.,취미오락,외식,M,30,202002,RED,휴식,321200000,11563,세종 .
38034880,세종,세종,.,취미오락,외식,F,20,202002,RED,활동,120500000,9568,세종 .
38034885,전남,세종,.,취미오락,종합쇼핑,M,30,202002,RED,활동,4713934,108,세종 .
38034890,울산,울산,남구,취미오락,인터넷게임,M,20,202002,WHITE,휴식,87900615,18255,울산 남구
38034894,울산,울산,남구,취미오락,종합쇼핑,M,20,202002,WHITE,활동,612900000,26660,울산 남구


In [ ]:
#주거지역과 결제지역이 서로 다른 지역을 여행지에서 이용했다고 가정
korea_yb_myterm = korea_yb_myterm[korea_yb_myterm['v1'] != korea_yb_myterm['v2']]

In [ ]:
#'gb3'열에서 여행으로 분류된 데이터만 이용
korea_yb_myterm = korea_yb_myterm[korea_yb_myterm['gb3'] == '여행'] ; korea_yb_myterm.head()

,v1,v2,v3,gb3,gb2,sex_ccd,cln_age_r,ta_ym,daw_ccd_r,apv_ts_dl_tm_r,vlm,usec,v4
38034952,인천,대전,동구,여행,교통,F,30,202002,WHITE,활동,228500000,6410,대전 동구
38034957,대구,대전,동구,여행,교통,M,30,202002,RED,휴식,50498910,2257,대전 동구
38034960,대구,대전,동구,여행,교통,M,30,202002,WHITE,활동,314100000,10117,대전 동구
38034974,서울,대전,동구,여행,교통,F,30,202002,WHITE,활동,2034000000,61375,대전 동구
38034975,강원,대전,동구,여행,교통,M,20,202002,RED,활동,133700000,9702,대전 동구


In [ ]:
#pivot_table을 이용해 index는 각 지역으로 잡고, column을 전국 시도단위로 잡고, value는 카드 이용량을 합치는 방식으로 계산했다.
korea_top = pd.pivot_table(korea_yb_myterm, index = 'v4', columns = 'v1', values = 'usec', aggfunc=np.sum)
korea_top

v1,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
v4,,,,,,,,,,,,,,,,,
강원 강릉시,NaN,191264.0,7238.0,10605.0,4460.0,6682.0,9358.0,8117.0,165733.0,2151.0,4123.0,38157.0,3277.0,4795.0,896.0,12052.0,22614.0
강원 고성군,NaN,58458.0,1265.0,1845.0,804.0,1586.0,2053.0,1446.0,54797.0,655.0,682.0,9538.0,704.0,743.0,184.0,2362.0,3696.0
강원 동해시,NaN,11930.0,2475.0,1302.0,299.0,816.0,764.0,1329.0,8685.0,145.0,369.0,2440.0,485.0,397.0,204.0,1040.0,1500.0
강원 삼척시,NaN,49403.0,1970.0,4395.0,747.0,2102.0,2290.0,1948.0,39176.0,886.0,1093.0,8989.0,746.0,827.0,75.0,2768.0,4670.0
강원 속초시,NaN,125618.0,3407.0,3769.0,1998.0,3084.0,4803.0,3828.0,119417.0,1483.0,1518.0,24929.0,1512.0,2030.0,569.0,6215.0,8914.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
충북 제천시,7980.0,13233.0,522.0,1502.0,228.0,770.0,1292.0,548.0,8117.0,274.0,205.0,2799.0,215.0,472.0,44.0,1504.0,NaN
충북 증평군,582.0,8767.0,294.0,723.0,134.0,259.0,7269.0,263.0,4460.0,3808.0,140.0,1117.0,169.0,612.0,10.0,5222.0,NaN
충북 진천군,413.0,5089.0,269.0,517.0,162.0,206.0,398.0,352.0,2054.0,284.0,119.0,1148.0,125.0,310.0,15.0,1315.0,NaN


In [ ]:
#위의 자료를 전치시켜 각 열의 값을 가지고 오름차순으로 상위 30개 지역을 파악하면서 여행자들의 카드 이용량이 적은 지역을 파악한다.
korea_top.T.sum().sort_values()[:30]

v4
전북 장수군     915.0
경북 영양군    1288.0
인천 동구     1450.0
경남 의령군    1456.0
전북 진안군    1559.0
충남 청양군    1616.0
경북 고령군    1986.0
전북 임실군    2255.0
경북 의성군    2963.0
경북 봉화군    3211.0
경남 함안군    3214.0
전남 강진군    3245.0
경남 산청군    3691.0
경북 예천군    3725.0
충북 영동군    3995.0
경남 함양군    4238.0
전남 장흥군    4274.0
전북 김제시    4862.0
충남 금산군    4949.0
강원 화천군    5004.0
경북 성주군    5048.0
경남 거창군    5270.0
광주 남구     5964.0
강원 양구군    6057.0
충북 보은군    6395.0
충남 계룡시    6431.0
강원 태백시    7098.0
경북 상주시    7189.0
전남 영광군    7346.0
경북 영천시    7356.0
dtype: float64

In [ ]:
korea_top.T.sum().sort_values()[-21:]

v4
서울 종로구       272417.0
경남 거제시       286510.0
강원 속초시       313094.0
강원 홍천군       457799.0
강원 강릉시       491522.0
경북 경주시       545488.0
전남 여수시       565007.0
서울 광진구       610292.0
강원 평창군       623530.0
부산 해운대구      649336.0
서울 송파구       748191.0
서울 중구        761206.0
인천 중구       1195871.0
부산 강서구      1209467.0
제주 서귀포시     1490749.0
서울 강서구      3412570.0
제주 제주시      6265727.0
경기 성남시      6363663.0
서울 서초구      7005364.0
서울 강남구      8205967.0
대전 동구      43378609.0
dtype: float64